## Import Dependencies


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import pandas as pd
import os
from model_training import ResNet, BottleNeck, ResNet50

## Create Dataset for local Directories

In [5]:
class CIFAKE_Dataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ['REAL', 'FAKE']
        
    def __len__(self):
        total_len = 0
        for cls in self.classes:
            class_dir = os.path.join(self.root_dir, cls)
            total_len += len(os.listdir(class_dir))
        return total_len
    
    def __getitem__(self, idx):
        for cls in self.classes:
            class_dir = os.path.join(self.root_dir, cls)
            if idx < len(os.listdir(class_dir)):
                img_name = os.listdir(class_dir)[idx]
                img_path = os.path.join(class_dir, img_name)
                image = Image.open(img_path)
                label = self.classes.index(cls)
                if self.transform:
                    image = self.transform(image)
                return image, label
            else:
                idx -= len(os.listdir(class_dir))

### Load the dataset

In [10]:
# Load the dataset from local directory without preprocessing
training_data = CIFAKE_Dataset(root_dir='data/CIFAKE Real and AI-Generated Images/train')
testing_data = CIFAKE_Dataset(root_dir='data/CIFAKE Real and AI-Generated Images/test')

In [11]:
# Defind the data loader
train_loader = DataLoader(training_data, batch_size=64, shuffle=True)
test_loader = DataLoader(testing_data, batch_size=64, shuffle=True)

In [ ]:
num_epochs = 100
net = ResNet50(10).to('cuda')
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=5)

In [ ]:
EPOCHS = 200
for epoch in range(EPOCHS):
    losses = []
    running_loss = 0
    for i, inp in enumerate(train_loader):
        inputs, labels = inp
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()
    
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i%100 == 0 and i > 0:
            print(f'Loss [{epoch+1}, {i}](epoch, minibatch): ', running_loss / 100)
            running_loss = 0.0

    avg_loss = sum(losses)/len(losses)
    scheduler.step(avg_loss)
            
print('Training Done')